In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# input directory path
input_dir = os.getcwd()
infilename = "data.csv"
# read input data file
in_df = pd.read_csv(os.path.join(input_dir, infilename))
# drop columns that contain only nans
#in_df = in_df.dropna(how = 'all', axis = 1)
#in_df = in_df.rename(columns = {'Gender (1 = male, 2 = female)':'Gender'})

In [3]:
class Preprocess_Dataframe:

    """ Preprocess input dataframe"""

    def __init__(self, in_df):
        self.in_df = in_df
    
    def remove_all_nans(self, axis_type: str):
        """
        Remove rows or columns containing all NaNs.

        Parameters
        ----------
        axis_type:  str
            'rows' or 'columns'
        """
        if axis_type == 'rows':
            self.in_df = self.in_df.dropna(how = 'all')
        else:
            self.in_df = self.in_df.dropna(how = 'all', axis = 1)
    
    def add_ids(self, id_vals = None):
        """
        Add id column.

        Parameters
        ----------
        in_df:  pd DataFrame
            Dataframe to operate on
        
        """
        if not id_vals:
            self.in_df['id'] = np.arange(1,self.in_df.shape[0]+1)
        else:
            self.in_df['id'] = id_vals 
    
    def rename_cols(self, old_names: list, new_names: list ):
        """
        Renames specified columns to specified new names.

        Parameters
        ----------
        old_names:  list of str
            names of columns to be renamed
        new_names:  list of str
            new column names
        """
        zipped_lists = zip(old_names, new_names)
        names_dict = dict(zipped_lists)
        self.in_df = self.in_df.rename(columns = names_dict)

    def prep_col_names(self, filter_by, suffix, split_by_num = None, suffix_2 = None):
        """
        Prep column names for use of wid_to_long method

        Parameters
        ----------
        filter_by: str
            substring to filter columns by
        prefix: str
            specify desired prefix
            for wide_to_long conversion
        """
        cols_to_prep = self.in_df.filter(like = filter_by, axis = 1).columns
        if not split_by_num:
            # NB: reduces flexibility, but can always use method overriding & child class if no longer
            # appropriate.
            prepped_col_names = ['_'.join([name.split()[0],suffix]) for name in cols_to_prep]
            self.rename_cols(cols_to_prep,prepped_col_names)
        else:
            new_names = ['numdays','enjoy','difficult']
            #prepped_col_names = ['_'.join([name, suffix]) for name in cols_to_prep[:split_by_num]]
            prepped_col_names = ['_'.join([name, suffix]) for name in new_names]
            prepped_col_names.extend(['_'.join([name, suffix_2]) for name in new_names])
            self.rename_cols(cols_to_prep,prepped_col_names)


    def reshape_data(self):
        """
        reshape df from wide to long format

        """
        col_names =  list(set([name.split('_')[0] for name in self.in_df.columns[3:-1]]))
        self.in_df_long = pd.wide_to_long(self.in_df, col_names,i = "id", j = "time", sep = '_').reset_index()
    
    def prepare_df(self):
        """
        Preprocess dataframe.
        """
        for index_name in ['rows','columns']:
            self.remove_all_nans(index_name)
        self.add_ids()
        gender_col = self.in_df.filter(like = 'Gender',axis = 1).columns
        self.rename_cols([gender_col[0], 'GHQ'], [gender_col[0].split()[0],'GHQ_0'])
        self.prep_col_names('Qual','1',3,'2')
        for filter_kw, prefix in [('baseline','0'),('day 10','1'),('day 30','2')]:
            self.prep_col_names(filter_kw,prefix)
        self.reshape_data()

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

class Data_Explorer:
    """
    Class for initial data exploration.
    """
    def __init__(self, in_df_long,group_col):
        self.in_df_long = in_df_long
        self.group_col = group_col
    
    def show_univariate_distributions(self, in_var_name, plot_type):
        pass

    def show_bivariate_distributions(self, var_names):
        pass
    
    def describe_baseline(self, var_names):
        """
        Describe the sample at baseline.
        """
        # Group = A
        # Group = B
        for group_name in self.in_df_long.Group.unique():
            describe_df = self.in_df_long.loc[(self.in_df_long.time==0) &
                                                    (self.in_df_long.Group==group_name),
                                                     var_names].describe()
            print(f"Group {group_name}: {describe_df}")
    
    def sample_describe(self,var_names: list, group_by = None):
        """
        describe the sample at baseline

        Parameters
        ----------
        var_names: list of str
            variables to describe
        group_by:    str
            if describing for more than one group,
            specify name of group col
        """
        if not group_by:
            self.describe_df = self.in_df_long.loc[self.in_df_long.time==0, var_names].describe()
        else:
            var_names.extend([group_by])
            self.describe_df = self.in_df_long.loc[:,var_names].groupby(group_by).describe()
        print(self.describe_df.T)
    
    def get_baseline_diffs(self, var_name):
        """
        Test for baseline differences.

        Parameters
        ----------
        var_name: str
            name of column to perform test on
        
        Returns
            statistic and p-value
        """
        _, norm_pval = stats.normaltest(self.in_df_long.loc[:,var_name])
        if norm_pval<0.05:
            non_parametric = 1
            print(f"Feature{var_name} is not normally distributed.\nUsing Mann Whitney U test.")
        else:
            non_parametric = None

        group_names = self.in_df_long[self.group_col].unique()
        samp1 = self.in_df_long.loc[(self.in_df_long[self.group_col] == group_names[0])&(self.in_df_long.time == 0),var_name]
        samp2 = self.in_df_long.loc[(self.in_df_long[self.group_col] == group_names[1])&(self.in_df_long.time == 0),var_name]

        if not non_parametric:
            stat,p_val = stats.ttest_ind(samp1, samp2, nan_policy = 'omit')
        else:
            stat,p_val = stats.mannwhitneyu(samp1, samp2)
        
        if p_val<0.05:
            print(f"\nGroups differ significantly at baseline on feature {var_name} (pval: {p_val}).")
        else:
            print(f"\nNo significant baseline differences detected on feature {var_name} (pval:{p_val}).\n")
        return stat, p_val



In [5]:
pp = Preprocess_Dataframe(in_df)
pp.prepare_df()

In [31]:
pp.in_df_long.loc[(pp.in_df_long.time==0)& (pp.in_df_long.Group=='A'),['GHQ','SWLS','PSS']].describe()

,GHQ,SWLS,PSS
count,38.000000,38.000000,38.000000
mean,3.578947,24.184211,16.684211
std,2.853462,5.727389,5.251024
min,0.000000,10.000000,5.000000
25%,1.000000,21.000000,14.000000
50%,3.500000,25.000000,16.000000
75%,5.750000,28.000000,19.750000
max,9.000000,35.000000,28.000000


In [34]:
de = Data_Explorer(pp.in_df_long,'Group')
de.describe_baseline(['GHQ','SWLS','PSS'])
#features = de.in_df_long.loc[de.in_df_long.time == 0,:].dropna(how = 'all',axis =1).columns
#features = [f for f in features if f not in ['Group','time','id']]
#for feature in features:
#    de.get_baseline_diffs(feature)

Group A:              GHQ       SWLS        PSS
count  38.000000  38.000000  38.000000
mean    3.578947  24.184211  16.684211
std     2.853462   5.727389   5.251024
min     0.000000  10.000000   5.000000
25%     1.000000  21.000000  14.000000
50%     3.500000  25.000000  16.000000
75%     5.750000  28.000000  19.750000
max     9.000000  35.000000  28.000000
Group B:              GHQ       SWLS        PSS
count  36.000000  34.000000  34.000000
mean    3.388889  24.617647  17.794118
std     3.227363   5.482837   5.563841
min     0.000000  11.000000   6.000000
25%     0.750000  21.000000  13.000000
50%     3.000000  27.000000  18.500000
75%     5.000000  28.000000  21.750000
max    10.000000  33.000000  28.000000


In [ ]:
de = Data_Explorer(in_df)
dem_describe = de.sample_describe(['Gender','Age'])

In [23]:
import warnings

class Data_Analyzer:
    def __init__(self, in_df_long,id_col):
        self.in_df_long = in_df_long
        self.id_col = id_col
    
    def scale_scores(self, col_names, scaler_type):
        """ 
        Scale questionnaire scores.

        Parameters
        ----------
        in_df:  pandas Dataframe
            Dataframe to operate on.
        col_names:  list
            list of column names to operate on.
        scaler_type:    scaler type to use
        """
        scaler = scaler_type

        for col in col_names:
            self.in_df_long[col+'_scaled'] = scaler.fit_transform(self.in_df_long[col].values.reshape(-1,1))

    def build_lmem(self, oc_name, predictor_names, interacts = None, r_slpe = None):
        """
        Use linear mixed effects model for analysis.
    
        Parameters
        ----------
        oc_name:    str
            outcome column name
        predictor_names:    list of str
            list of column names to use as predictors
        interacts:  
            Flag to indicate whether to construct the formula
            using interactions ( pred1*pred2) or not.
            Any value is fine, as the code only checks whether
            this is None or not.
        """
        if not interacts:
            rh_side = '+'.join(predictor_names)
        else:
            rh_side = '*'.join(predictor_names)
        self.formula = ''.join([oc_name,'~',rh_side])
        if not r_slpe: 
            self.model_lmem = smf.mixedlm(self.formula, self.in_df_long,
                                        groups=self.id_col,missing = 'drop').fit()
        else:
            self.model_lmem = smf.mixedlm(self.formula, self.in_df_long, groups=self.id_col, 
                                    re_formula = ''.join(['~',r_slpe]),missing = 'drop').fit()
        print(f"\nModel results:\n{self.model_lmem.summary()}")
    
    def build_gee(self, oc_name, predictor_names, fam, cov_type, interacts = None):
        """
        Use GEE approach for analysis.
        
        Parameters
        ----------
        oc_name:    str
            outcome column name
        predictor_names:    list of str
            list of column names to use as predictors
        interacts:  
            Flag to indicate whether to construct the formula
            using interactions ( pred1*pred2) or not.
            Any value is fine, as the code only checks whether
            this is None or not.
        fam:
            Mean response structure
        cov_type:
            covariance structure
        """
        if not interacts:
            rh_side = '+'.join(predictor_names)
        else:
            rh_side = '*'.join(predictor_names)
        self.formula = ''.join([oc_name,'~',rh_side])
        try:
            self.model_gee = smf.gee(self.formula,self.id_col, self.in_df_long,
                            cov_struct = cov_type, family = fam,
                            missing = 'drop').fit()
        except ValueError:
            warnings.warn("Covariance structure changed to Independence.",
                            category = UserWarning)
            self.model_gee = smf.gee(self.formula,self.id_col, self.in_df_long,
                            cov_struct = sm.cov_struct.Independence(),
                            family = fam,
                            missing = 'drop').fit()
        print(self.model_gee.summary2())

In [24]:
da = Data_Analyzer(pp.in_df_long,"id")
da.build_lmem("PSS",["C(Group,Treatment('B'))", "C(time,Treatment(0))"],interacts = 1,r_slpe = "time")
da.build_gee("PSS",["C(Group,Treatment('B'))", "C(time,Treatment(0))"],sm.families.Gaussian(), sm.cov_struct.Autoregressive(),interacts = 1)


Model results:
                               Mixed Linear Model Regression Results
Model:                           MixedLM                Dependent Variable:                PSS      
No. Observations:                198                    Method:                            REML     
No. Groups:                      72                     Scale:                             8.3824   
Min. group size:                 1                      Log-Likelihood:                    -566.7706
Max. group size:                 3                      Converged:                         Yes      
Mean group size:                 2.8                                                                
----------------------------------------------------------------------------------------------------
                                                         Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------------------------------------
Interc

C:\Users\Luzia T\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\cov_struct.py:796: FutureWarning: grid=True will become default in a future version
  warnings.warn(


In [20]:
da.model_gee.summary2()

AttributeError: 'Data_Analyzer' object has no attribute 'model_gee'